In [1]:
from IPython.display import set_matplotlib_formats
from scipy import interpolate
from scipy import optimize
from scipy.optimize import curve_fit
from scipy.stats import gaussian_kde
from decimal import *
# from tabulate import tabulate
from astropy.table import QTable
from astropy.io import fits
from astropy.io import ascii
import math
import numpy as np
np.float = np.float64
np.int = np.int64
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
import scipy.stats as st
from astropy import constants as c
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)
%matplotlib inline
set_matplotlib_formats('svg')
import matplotlib.cm
import glob
import h5py
import os
import time as ostime
import scipy.interpolate as interpolate
from multiprocessing import Process, Queue
import multiprocessing
import matplotlib.cm     as cm
from scipy import optimize
from scipy import stats
import datetime
import argparse
import unyt
from unyt import unyt_array
import yt
# yt.toggle_interactivity()
from yt.units import kpc
from yt.units import dimensions
import pyxsim
from pyxsim.source_models import CIESourceModel

# plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = [12, 8]

# colormap = plt.cm.get_cmap('brg') #or arbre

/tmp/ipykernel_929193/1414713785.py:24: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')
soxs : [WARNING  ] 2023-10-12 15:23:12,419 Setting 'soxs_data_dir' to /home/rongjun/.cache/soxs for this session. Please update your configuration if you want it somewhere else.


In [2]:
# Import values
gamma = 5/3
boltzmann_constant_cgs = (c.k_B.to(u.erg / u.K)).value*unyt.erg/unyt.K
start_time = ostime.time()
m_p_cgs = (c.m_p.to(u.g)).value*unyt.g

In [3]:
# unit_base = {
#     "length": (1.0, "kpc"),
#     "velocity": (1.0, "km/s"),
#     "mass": (1.0, "Msun"),
#     "density": (1.0, "g/cm**3")
# }

ds_plt1555000 = yt.load("plt1555000", default_species_fields='ionized')

yt : [INFO     ] 2023-10-12 15:23:12,959 Parameters: current_time              = 4858047908532521.0
yt : [INFO     ] 2023-10-12 15:23:12,960 Parameters: domain_dimensions         = [ 128  128 1024]
yt : [INFO     ] 2023-10-12 15:23:12,961 Parameters: domain_left_edge          = [ 0.0000e+00  0.0000e+00 -1.2072e+22]
yt : [INFO     ] 2023-10-12 15:23:12,961 Parameters: domain_right_edge         = [3.0180e+21 3.0180e+21 1.2072e+22]


In [4]:
print(ds_plt1555000.field_list)

[('boxlib', 'gasDensity'), ('boxlib', 'gasEnergy'), ('boxlib', 'gasInternalEnergy'), ('boxlib', 'scalar_0'), ('boxlib', 'scalar_1'), ('boxlib', 'scalar_2'), ('boxlib', 'x-GasMomentum'), ('boxlib', 'y-GasMomentum'), ('boxlib', 'z-GasMomentum')]


In [5]:
# Define the level of refinement. For base level, use level=0.
level = 0

# Find the domain dimensions of the dataset at the specified level of refinement.
dims = ds_plt1555000.domain_dimensions * ds_plt1555000.refine_by**level

# Use the covering_grid method with the dataset's domain left and right edges.
covering_grid = ds_plt1555000.covering_grid(level=level, left_edge=ds_plt1555000.domain_left_edge, dims=dims)
gasDensity = covering_grid[("boxlib", "gasDensity")]
gasInternalEnergy = covering_grid[("boxlib", "gasInternalEnergy")]

time = ds_plt1555000.current_time.to('Myr')

def _gasDensity(field, data):
    density = gasDensity*unyt.g/unyt.cm**3
    return density

ds_plt1555000.add_field(('gas', 'density'), 
                        function=_gasDensity, units='g/cm**3', sampling_type='cell', force_override=True)

def _gasInternalEnergy(field, data):
    InternalEnergy = gasInternalEnergy*unyt.erg
    return InternalEnergy

ds_plt1555000.add_field(('gas', 'gasInternalEnergy'), 
                        function=_gasInternalEnergy, units='erg', sampling_type='cell', force_override=True)

In [6]:
print(gasDensity.shape)
print(gasInternalEnergy.shape)
print(time)

(128, 128, 1024)
(128, 128, 1024)
153.9422487303382 Myr


In [7]:
#Grackle Cooling Table
file = 'grackle_data_files/input/CloudyData_UVB=HM2012.h5'
grackle = h5py.File(file)
array = grackle['CoolingRates/Primordial/MMW'][()]
table = array[:,0,:]
table_nH   = np.logspace(-6, 4, array.shape[0])
table_temp = np.logspace(1,  9, array.shape[2])

bins = 100
egas_arr = np.logspace(-16., -5., bins)
nH_arr   = np.logspace(-6.0, 4.0, int(bins))
logrho_arr = np.log10(nH_arr[:-1])
logEgas_arr = np.log10(egas_arr[:-1])

#Set up the interpolator

T = np.zeros((egas_arr.shape[0],nH_arr.shape[0]))
i=0
for egas in egas_arr:
    j=0
    for nH in nH_arr:
        C = (gamma - 1.) * egas / (boltzmann_constant_cgs.v*nH)
        minT = C*np.amin(table)
        maxT = C*np.amax(table)
        def func(T):
            mu = interpolate.interp2d(table_temp, table_nH, table,\
                              kind='linear', copy=True, bounds_error=False, fill_value=None)
            return C*mu(T,nH)[0] - T

        T[i,j] = optimize.toms748(func, minT, maxT)
        j+=1
    i+=1

In [8]:
# import sys
# import os
# import argparse

# # Mock the command line arguments
# sys.argv = ['ipykernel_launcher.py', '--file', os.getcwd()]

lev = 0
home = os.getcwd()

# parser = argparse.ArgumentParser(description='Optional app description')
# parser.add_argument('--file', type=str, help='Filename to be analysed')
# args = parser.parse_args()

class Data:
    fac = 1
    lev = 0
    file = ''
    dom_min = [0.0, 0.0, 0.0]
    def getData(file):
        dom_min = [0.0, 0.0, 0.0]
        ds   = yt.load(file)
        data = ds.covering_grid(level=lev, left_edge=dom_min, dims=ds.domain_dimensions * fac)
        density = np.array(data['gasDensity'])
        time = ds.current_time.to('Myr')
        Egas = np.array(data["gasInternalEnergy"])
        return density, Egas, time

def getdomain(file):
    infile = open(file)
    lines = infile.readlines()
    dom_range = np.zeros((2,3))
    ncell = np.zeros(3)
    dom_min = [0.0,0.0,0.0]
    dom_min[0] = float(lines[3].split()[2])
    dom_min[1] = float(lines[3].split()[3])
    dom_min[2] = float(lines[3].split()[4])
    
    dom_max = [0.0,0.0,0.0]
    dom_max[0] = float(lines[4].split()[2])
    dom_max[1] = float(lines[4].split()[3])
    dom_max[2] = float(lines[4].split()[4])

    ncell[0]=int(lines[15].split()[2])
    ncell[1]=int(lines[15].split()[3])
    ncell[2]=int(lines[15].split()[4])
    
    return dom_min, dom_max, ncell

# Extracted information from the log
domain_left_edge = ds_plt1555000.domain_left_edge.to('kpc')
domain_right_edge = ds_plt1555000.domain_right_edge.to('kpc')
domain_dimensions = ds_plt1555000.domain_dimensions
domain_center = ds_plt1555000.domain_center.to('kpc')

# Compute ranges
x_range = (domain_left_edge[0], domain_right_edge[0])
y_range = (domain_left_edge[1], domain_right_edge[1])
z_range = (domain_left_edge[2], domain_right_edge[2])

# Extract number of cells
n_cell_x = domain_dimensions[0]
n_cell_y = domain_dimensions[1]
n_cell_z = domain_dimensions[2]
n_cell_total = n_cell_x * n_cell_y * n_cell_z

print(f"x_range: {x_range}")
print(f"y_range: {y_range}")
print(f"z_range: {z_range}")
print(f"Total number of cells: {n_cell_total}")


# if(lev==0):
#     fac = 1
# else:
#     fac = 2 * lev
# Data.fac = fac

# infile = os.path.join(home, 'plt1555000')
# dom_min, dom_max, ncells = getdomain(infile)
# zrange = np.linspace(dom_min[2], dom_max[2], (fac*int(ncells[2])))

# outhome = os.getcwd()
# outputfile_name = os.path.join(outhome, 'plt1555000' + filename.split('plt')[1] + '.h5')



x_range: (unyt_quantity(0., 'kpc'), unyt_quantity(0.97806719, 'kpc'))
y_range: (unyt_quantity(0., 'kpc'), unyt_quantity(0.97806719, 'kpc'))
z_range: (unyt_quantity(-3.91226876, 'kpc'), unyt_quantity(3.91226876, 'kpc'))
Total number of cells: 16777216


In [9]:
# ds   = yt.load(filename)
# data = ds.covering_grid(level=lev, left_edge=dom_min, dims=ds.domain_dimensions * fac)
timestep = ds_plt1555000.current_time.to('Myr')

rho = gasDensity
egas0 = gasInternalEnergy

rho0 = rho/m_p_cgs


logrho_arr = np.log10(nH_arr[:-1])
logrho = np.log10(rho0)
delta_rho = logrho_arr[1] - logrho_arr[0]
idxrho = (np.floor((logrho - np.amin(logrho_arr))/delta_rho)).astype('int')

logEgas_arr = np.log10(egas_arr[:-1])
logEgas = np.log10(egas0)
delta_egas = logEgas_arr[1] - logEgas_arr[0]
idxegas = (np.floor((logEgas-np.amin(logEgas_arr))/delta_egas)).astype('int')


wgt_rho  = (logrho - (np.amin(logrho_arr) + delta_rho*idxrho))/delta_rho
wgt_egas = (logEgas - (np.amin(logEgas_arr) + delta_egas*idxegas))/delta_egas

temp = (1.-wgt_rho)*(1.-wgt_egas)* T[tuple(idxegas)  , tuple(idxrho)]   +\
        wgt_rho *    wgt_egas * T[tuple(idxegas+1), tuple(idxrho+1)] +\
    (1. -wgt_rho)*    wgt_egas * T[tuple(idxegas+1), tuple(idxrho)]   +\
        wgt_rho *(1.-wgt_egas)* T[tuple(idxegas)  , tuple(idxrho+1)]  

# rho0 = rho/m_p_cgs


# logrho_arr = np.log10(nH_arr[:-1])
# logrho = np.log10(rho0)
# delta_rho = logrho_arr[1] - logrho_arr[0]
# idxrho = (np.floor((logrho - np.amin(logrho_arr))/delta_rho)).astype('int')

# logEgas_arr = np.log10(egas_arr[:-1])
# logEgas = np.log10(egas0)
# delta_egas = logEgas_arr[1] - logEgas_arr[0]
# idxegas = (np.floor((logEgas-np.amin(logEgas_arr))/delta_egas)).astype('int')


# wgt_rho  = (logrho - (np.amin(logrho_arr) + delta_rho*idxrho))/delta_rho
# wgt_egas = (logEgas - (np.amin(logEgas_arr) + delta_egas*idxegas))/delta_egas

# Temp = (1.-wgt_rho) * (1.-wgt_egas)* T[tuple(idxegas)  , tuple(idxrho)]   +\
#             wgt_rho  *    wgt_egas  * T[tuple(idxegas+1), tuple(idxrho+1)] +\
#         (1. -wgt_rho) *    wgt_egas  * T[tuple(idxegas+1), tuple(idxrho)]   +\
#             wgt_rho  * (1.-wgt_egas)* T[tuple(idxegas)  , tuple(idxrho+1)]  


# outhome = '/scratch/pawsey0807/aditi/quokka_devbranch/Analysis/data/'
# outputfile_name = os.path.join(outhome,build,run, 'temperature_' + filename.split('plt')[1] + '.h5')


# hfo = h5py.File(outputfile_name, 'w')
# hfo.create_dataset('Temp'       , data=temp)
# hfo.create_dataset('Timestep', data=timestep)
# hfo.close()
# print("--------Written file------->>",f)

In [10]:
def _temp_field(field, data):
#     reshaped_temp = temp.reshape(ds_plt1555000.domain_dimensions[::1])
    reshaped_temp = temp
    return data.ds.arr(reshaped_temp, "K")

ds_plt1555000.add_field(("gas", "temperature"), function=_temp_field, units="K", sampling_type='cell', 
                        force_override=True)

ad_plt1555000 = ds_plt1555000.all_data()
# print(ad_plt1555000[("gas", "temp")])

In [11]:
all_data = ds_plt1555000.all_data()
print(all_data[("gas", "temperature")].shape)
print(ds_plt1555000.domain_dimensions)

(128, 128, 1024)
[ 128  128 1024]


In [12]:
# Calculate the size of the domain in each dimension
domain_size = domain_right_edge - domain_left_edge

# Calculate the cell width in each dimension
cell_widths = domain_size / domain_dimensions

# Assuming cubic cells, take the cell width from one dimension
dx = (cell_widths[0]).to(unyt.cm)
dy = (cell_widths[1]).to(unyt.cm)
dz = (cell_widths[2]).to(unyt.cm)

ad_plt1555000 = ds_plt1555000.all_data()
def _emission_measure(field, data):
    nH = ad_plt1555000[('gas', 'density')]/m_p_cgs
#     ne = 1.2 * nH
    return nH **2 * dx * dy * dz  # .d extracts the value as a float

ds_plt1555000.add_field(('gas', 'emission_measure'), 
                        function=_emission_measure, units='cm**-3', sampling_type='cell', force_override=True)

In [13]:
T_min = np.min(temp)  # minimum temperature for which we'll emit X-rays
T_max = np.max(temp)  # maximum temperature for which we'll emit X-rays
e_min = (c.k_B*T_min*u.K).to(u.keV)
e_max = (c.k_B*T_max*u.K).to(u.keV)
# n_cells = 1e4  # the number of cells to "sample" from the dataset

ad_plt1555000 = ds_plt1555000.all_data()
def _gasMass(field, data):
    mass = ad_plt1555000[('gas', 'density')]*dx*dy*dz
    return mass

ds_plt1555000.add_field(('gas', 'mass'), 
                        function=_gasMass, units='g', sampling_type='cell', force_override=True)

# source_model = CIESourceModel("apec", 
#                               e_min, e_max, 
#                               Zmet=1.0, nbins=1024, binscale="log")

source_model = pyxsim.CIESourceModel("apec", e_min, e_max, nbins=1024, Zmet=1.0, binscale="log")
xray_fields = source_model.make_source_fields(ds_plt1555000, e_min, e_max, force_override=True)
# source_model = pyxsim.CIESourceModel("spex", 0.5, 7.0, nbins=1024, Zmet=1.0, binscale="log")
# xray_fields = source_model.make_source_fields(ds_plt1555000, 0.5, 7.0, force_override=True)
print(xray_fields)

pyxsim : [INFO     ] 2023-10-12 15:23:30,205 kT_min = 0.025 keV
pyxsim : [INFO     ] 2023-10-12 15:23:30,206 kT_max = 64 keV
pyxsim : [INFO     ] 2023-10-12 15:23:30,207 Using emission measure field '('gas', 'emission_measure')'.
pyxsim : [INFO     ] 2023-10-12 15:23:30,207 Using temperature field '('gas', 'temperature')'.


Preparing spectrum table :   0%|          | 0/138 [00:00<?, ?it/s]

[('gas', 'xray_emissivity_2.6831870459202334e-06_2926588.545703103_keV'), ('gas', 'xray_luminosity_2.6831870459202334e-06_2926588.545703103_keV'), ('gas', 'xray_photon_emissivity_2.6831870459202334e-06_2926588.545703103_keV')]


In [14]:
# source_model = CIESourceModel("apec", 
#                               e_min, e_max, 
#                               Zmet=1.0, nbins=1024, binscale="log")
# source_model = pyxsim.CIESourceModel("spec", 0.05, 11.0, 1000, 0.1, binscale="log")
region = ds_plt1555000.region(domain_center, domain_left_edge, domain_right_edge)

exp_time = (300.0, "ks")  # exposure time
area = (1000.0, "cm**2")  # collecting area
redshift = 0.05

# n_photons, n_cells  = pyxsim.make_photons("my_photons", ds_plt1555000, redshift, area, exp_time, source_model)

In [15]:
test_region = ds_plt1555000.region(
    domain_center, domain_center-dx, domain_center+dx)
print(test_region)

YTRegion (plt1555000): , center=[1.509e+21 1.509e+21 0.000e+00] cm, left_edge=[ 1.48542188e+21  1.48542188e+21 -2.35781250e+19] cm, right_edge=[1.53257813e+21 1.53257813e+21 2.35781250e+19] cm


In [16]:
sp_plt1555000 = ds_plt1555000.sphere("c", (10, "kpc"))
print(sp_plt1555000)

YTSphere (plt1555000): , center=[1.509e+21 1.509e+21 0.000e+00] cm, radius=3.085677580962325e+22 cm


In [17]:
ad_plt1555000.ds.derived_field_list

[('boxlib', 'cell_volume'),
 ('boxlib', 'dx'),
 ('boxlib', 'dy'),
 ('boxlib', 'dz'),
 ('boxlib', 'gasDensity'),
 ('boxlib', 'gasEnergy'),
 ('boxlib', 'gasInternalEnergy'),
 ('boxlib', 'path_element_x'),
 ('boxlib', 'path_element_y'),
 ('boxlib', 'path_element_z'),
 ('boxlib', 'scalar_0'),
 ('boxlib', 'scalar_1'),
 ('boxlib', 'scalar_2'),
 ('boxlib', 'volume'),
 ('boxlib', 'x'),
 ('boxlib', 'x-GasMomentum'),
 ('boxlib', 'y'),
 ('boxlib', 'y-GasMomentum'),
 ('boxlib', 'z'),
 ('boxlib', 'z-GasMomentum'),
 ('gas', 'cell_volume'),
 ('gas', 'density'),
 ('gas', 'dx'),
 ('gas', 'dy'),
 ('gas', 'dz'),
 ('gas', 'emission_measure'),
 ('gas', 'gasInternalEnergy'),
 ('gas', 'mass'),
 ('gas', 'path_element_x'),
 ('gas', 'path_element_y'),
 ('gas', 'path_element_z'),
 ('gas', 'temperature'),
 ('gas', 'volume'),
 ('gas', 'x'),
 ('gas', 'xray_emissivity_2.6831870459202334e-06_2926588.545703103_keV'),
 ('gas', 'xray_luminosity_2.6831870459202334e-06_2926588.545703103_keV'),
 ('gas',
  'xray_photon_emis

In [18]:
ad_plt1555000[('index','x')].shape

(16777216,)

In [19]:
# sp_plt1555000 = ds_plt1555000.sphere("c", (10, "kpc"))
sp_plt1555000[('index','x')].shape

(16777216,)

In [20]:
# %pdb

In [21]:
# ds_plt1555000.force_periodicity()

ad_plt1555000 = ds_plt1555000.all_data()
buffer = 0.05 * dx  # this is a 5% buffer, but you can adjust as needed
slightly_smaller_left_edge = domain_left_edge + buffer
slightly_smaller_right_edge = domain_right_edge - buffer
slightly_smaller_region = ds_plt1555000.region(
    domain_center, slightly_smaller_left_edge, slightly_smaller_right_edge)
sp_plt1555000 = ds_plt1555000.sphere("c", (10, "kpc"))
region = ds_plt1555000.region(domain_center, domain_left_edge, domain_right_edge)

import traceback

try:
    n_photons, n_cells = pyxsim.make_photons("plt1555000_photon_list", 
                              data_source=sp_plt1555000, 
                                  redshift=redshift, area=area, 
                              exp_time=exp_time, source_model=source_model)
except Exception as e:
    print("An error occurred:", e)
    traceback.print_exc()


pyxsim : [INFO     ] 2023-10-12 15:24:41,381 Cosmology: h = 0.71, omega_matter = 0.27, omega_lambda = 0.73
pyxsim : [INFO     ] 2023-10-12 15:24:41,382 Using emission measure field '('gas', 'emission_measure')'.
pyxsim : [INFO     ] 2023-10-12 15:24:41,382 Using temperature field '('gas', 'temperature')'.


Preparing spectrum table :   0%|          | 0/138 [00:00<?, ?it/s]

Processing cells/particles :   0%|          | 0/268435456 [00:00<?, ?it/s]

An error occurred: index 1134054 is out of bounds for axis 0 with size 1048576


Traceback (most recent call last):
  File "/tmp/ipykernel_929193/3620299636.py", line 15, in <module>
    n_photons, n_cells = pyxsim.make_photons("plt1555000_photon_list",
  File "/home/rongjun/.local/lib/python3.8/site-packages/pyxsim/photon_list.py", line 416, in make_photons
    pos = chunk[p_fields[i]][idxs].to_value("kpc")
  File "/home/rongjun/.local/lib/python3.8/site-packages/unyt/array.py", line 1722, in __getitem__
    ret = super(unyt_array, self).__getitem__(item)
IndexError: index 1134054 is out of bounds for axis 0 with size 1048576


In [22]:
# Calculate the X-ray luminosity
energies = photons["energy"]
L_X = np.sum(energies)

print(L_X)

NameError: name 'photons' is not defined